In [14]:
import pandas as pd
df = pd.read_csv('spam.csv')
df.head()

,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
2,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
3,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
4,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN


In [15]:
dataset = pd.DataFrame()

In [20]:
dataset['tags'] = df[df.columns[0]]

In [22]:
dataset['text'] = df[df.columns[1]]

In [23]:
dataset.head()

,tags,text
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [25]:
dataset.tags.value_counts()

ham     4824
spam     747
Name: tags, dtype: int64

In [26]:
dataset['labels'] = dataset['tags'].map({'spam':1, 'ham':0})

In [27]:
dataset.head()

,tags,text,labels
0,ham,Ok lar... Joking wif u oni...,0
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
2,ham,U dun say so early hor... U c already then say...,0
3,ham,"Nah I don't think he goes to usf, he lives aro...",0
4,spam,FreeMsg Hey there darling it's been 3 week's n...,1


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(dataset['text'], dataset['labels'], test_size=0.2)

In [37]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [48]:
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocess = bert_preprocess(input_text)
outputs = bert_model(preprocess)

l = tf.keras.layers.Dropout(0.2)(outputs['pooled_output'])
# you can change the fully-connected layers according to yourself 

# l = tf.keras.layers.Dense(128,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(l)
# l = tf.keras.layers.Dropout(0.4)(l)
# l = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(l)
# l = tf.keras.layers.Dropout(0.2)(l)
l = tf.keras.layers.Dense(1, activation='sigmoid')(l)

model = tf.keras.Model(inputs = [input_text], outputs=[l])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_word_ids':   0           ['input_9[0][0]']                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
140/140 [==============================] - 93s 647ms/step - loss: 0.3255 - accuracy: 0.8687
Epoch 2/5
140/140 [==============================] - 91s 653ms/step - loss: 0.2419 - accuracy: 0.8923
Epoch 3/5
140/140 [==============================] - 92s 656ms/step - loss: 0.2075 - accuracy: 0.9140
Epoch 4/5
140/140 [==============================] - 92s 656ms/step - loss: 0.1806 - accuracy: 0.9311
Epoch 5/5
140/140 [==============================] - 92s 656ms/step - loss: 0.1678 - accuracy: 0.9356


In [51]:
model.evaluate(X_test, y_test)

35/35 [==============================] - 24s 643ms/step - loss: 0.1609 - accuracy: 0.9561


[0.16093286871910095, 0.9560537934303284]

In [71]:
pred = model.predict(["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"])

1/1 [==============================] - 0s 53ms/step


In [72]:
if pred[0][0] > 0.5:
    print('spam')

spam


In [ ]:
import pickle as pkl
save_model = pkl.dump(model)